<a href="https://colab.research.google.com/github/yudahendriawan/google-colab-projects/blob/general/converting_dataset_to_TFDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)

In [4]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size = 0.2)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


34521 train examples
8631 validation examples
10788 test examples


In [6]:
#membuat kolom target
# 0 = low; 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)

#drop unused columns
df = df.drop(columns=['price'])

In [7]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z,target
0,1,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,0
1,2,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,0
2,3,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,0
3,4,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,0
4,5,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,0


In [8]:
# cara untuk membuat dataset tf.data dari pandas dataframe

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = df.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

batch_size = 10
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [9]:
example_batch = next(iter(train_ds))[0]

def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [10]:
# numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[1.27]
 [0.32]
 [1.12]
 [1.13]
 [0.43]
 [0.32]
 [1.03]
 [1.  ]
 [0.7 ]
 [1.22]]


In [11]:
#bucketized column

carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1,2])
demo(carat_buckets)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [12]:
# categorical

color_type = feature_column.categorical_column_with_vocabulary_list(
    'color', ['E','I','J','D','H','G','F']
)

color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]]


In [13]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[ 0.4836634  -0.29662997  0.36289015  0.1472967  -0.29587388 -0.61677915]
 [ 0.01984874 -0.51851773  0.12086753 -0.05774143  0.01898976 -0.33796024]
 [-0.20498033 -0.10061408  0.67250687 -0.7151074  -0.4849288  -0.3678922 ]
 [ 0.4836634  -0.29662997  0.36289015  0.1472967  -0.29587388 -0.61677915]
 [-0.10762403  0.05424543 -0.14557298  0.36460894  0.43083188  0.20205358]
 [ 0.01984874 -0.51851773  0.12086753 -0.05774143  0.01898976 -0.33796024]
 [-0.20498033 -0.10061408  0.67250687 -0.7151074  -0.4849288  -0.3678922 ]
 [ 0.4836634  -0.29662997  0.36289015  0.1472967  -0.29587388 -0.61677915]
 [-0.20498033 -0.10061408  0.67250687 -0.7151074  -0.4849288  -0.3678922 ]
 [-0.08126739 -0.39052007 -0.16807751  0.24065956  0.42321774 -0.21418571]]


In [14]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


In [15]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

Instructions for updating:
Use `tf.keras.layers.experimental.preprocessing.HashedCrossing` instead for feature crossing when preprocessing data to train a Keras model.


[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [16]:
#Pilih feature column mana yang akan digunakan
feature_columns = []
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [17]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)


Epoch 1/10
1686/1686 [==============================] - 16s 8ms/step - loss: 0.2090 - accuracy: 0.9099 - val_loss: 0.1657 - val_accuracy: 0.9248
Epoch 2/10
1686/1686 [==============================] - 11s 6ms/step - loss: 0.1580 - accuracy: 0.9339 - val_loss: 0.1432 - val_accuracy: 0.9382
Epoch 3/10
1686/1686 [==============================] - 12s 7ms/step - loss: 0.1551 - accuracy: 0.9349 - val_loss: 0.1454 - val_accuracy: 0.9387
Epoch 4/10
1686/1686 [==============================] - 12s 7ms/step - loss: 0.1506 - accuracy: 0.9361 - val_loss: 0.1483 - val_accuracy: 0.9385
Epoch 5/10
1686/1686 [==============================] - 11s 6ms/step - loss: 0.1493 - accuracy: 0.9360 - val_loss: 0.1434 - val_accuracy: 0.9383
Epoch 6/10
1686/1686 [==============================] - 12s 7ms/step - loss: 0.1479 - accuracy: 0.9370 - val_loss: 0.1401 - val_accuracy: 0.9382
Epoch 7/10
1686/1686 [==============================] - 12s 7ms/step - loss: 0.1469 - accuracy: 0.9357 - val_loss: 0.1397 - val_ac